# 전력수요 예측 특화 LLM - SFT (Supervised Fine-Tuning)

## 접근법
1. **GPT API**로 전력수요 문서 → Q&A 쌍 생성
2. **SFT (지도학습)**으로 Instruction-tuned 모델 파인튜닝
3. 자기지도학습(DAPT) ❌, 지도학습(SFT) ✅

## 왜 SFT?
- Instruct 모델의 질문-답변 능력 유지
- 전력수요 도메인 지식 학습
- DAPT는 instruction 능력을 깨뜨릴 위험

In [ ]:
# 필요한 라이브러리 설치
# !pip install transformers datasets torch accelerate bitsandbytes peft trl sentencepiece openai

In [1]:
import os
import json
from pathlib import Path
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig
)
from datasets import Dataset, load_dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

/home/user/de-llama/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA device: NVIDIA RTX A6000
CUDA memory: 51.04 GB


## 1. Q&A 데이터셋 생성 (GPT API 사용)

먼저 `generate_qa_dataset.py` 스크립트를 실행하여 Q&A 데이터셋을 생성합니다.

In [2]:
# OpenAI API 키 설정 확인
import os

if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  OPENAI_API_KEY가 설정되지 않았습니다.")
    print("\n다음 명령어로 설정하세요:")
    print("export OPENAI_API_KEY='your-api-key-here'")
else:
    print("✓ OPENAI_API_KEY 설정됨")

✓ OPENAI_API_KEY 설정됨


In [3]:
# Q&A 데이터셋 생성 스크립트 실행
# 터미널에서 실행:
# python generate_qa_dataset.py

# 또는 여기서 직접 실행 (테스트용, 5개 파일만)
# %run generate_qa_dataset.py

## 2. 생성된 Q&A 데이터셋 로딩 및 확인

In [4]:
# SFT 데이터셋 로딩
dataset_file = "sft_dataset.jsonl"

if not Path(dataset_file).exists():
    print(f"❌ {dataset_file} 파일이 없습니다.")
    print("먼저 generate_qa_dataset.py를 실행하여 데이터셋을 생성하세요.")
else:
    # JSONL 파일 로딩
    with open(dataset_file, 'r', encoding='utf-8') as f:
        sft_data = [json.loads(line) for line in f]
    
    print(f"✓ 총 {len(sft_data)}개 Q&A 쌍 로딩 완료")
    
    # 샘플 확인
    print("\n=== 샘플 데이터 ===\n")
    for i, sample in enumerate(sft_data[:3], 1):
        print(f"[{i}] 질문: {sample['messages'][0]['content']}")
        print(f"    답변: {sample['messages'][1]['content'][:150]}...\n")

✓ 총 1139개 Q&A 쌍 로딩 완료

=== 샘플 데이터 ===

[1] 질문: 2019년 12월의 최대부하는 얼마인가요?
    답변: 2019년 12월의 최대부하는 8,730만kW로 예측되었습니다....

[2] 질문: What was the maximum load in December 2018?
    답변: The maximum load in December 2018 was 8,622 만kW....

[3] 질문: 2019년 12월의 주별 최대 전력 수요는 어떻게 되나요?
    답변: 2019년 12월의 주별 최대 전력 수요는 1주차 8,240만 kW, 2주차 8,420만 kW, 3주차 8,540만 kW, 4주차 8,730만 kW로 증가하는 추세를 보였습니다....



In [5]:
# Dataset 객체로 변환
dataset = load_dataset('json', data_files=dataset_file, split='train')

# Train/Test 분할
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"Train set: {len(dataset['train'])}개")
print(f"Test set: {len(dataset['test'])}개")

Train set: 1025개
Test set: 114개


## 3. 모델 및 토크나이저 로딩

### 추천 모델:
1. **EEVE-Korean-10.8B**: `yanolja/EEVE-Korean-Instruct-10.8B-v1.0`
2. **Qwen2.5-7B**: `Qwen/Qwen2.5-7B-Instruct`

In [6]:
# 모델 선택
model_name = "Qwen/Qwen2.5-7B-Instruct"  # 또는 "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

print(f"Loading model: {model_name}")

Loading model: Qwen/Qwen2.5-7B-Instruct


In [7]:
# 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# padding token 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✓ Tokenizer loaded")
print(f"  Vocab size: {len(tokenizer)}")
print(f"  Pad token: {tokenizer.pad_token}")

✓ Tokenizer loaded
  Vocab size: 151665
  Pad token: <|endoftext|>


In [8]:
# 4bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# 모델 로딩
model_name = "./model_weights"  # <-- 여기가 핵심입니다!

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    local_files_only=True, # 인터넷 연결 안 하고 로컬 파일만 쓰겠다는 강력한 옵션
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

print(f"✓ Model loaded")
print(f"  Device: {model.device}")

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.39s/it]

✓ Model loaded
  Device: cuda:0


In [9]:
print(model.dtype)   # torch.float16 이어야 함


torch.float16


## 4. 학습 전 모델 테스트

In [10]:
def test_model(prompt, max_new_tokens=200):
    """
    모델 테스트 함수
    """
    # Chat template 적용
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # 입력 부분 제거하고 생성된 부분만 추출
    generated = outputs[0][inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(generated, skip_special_tokens=True)
    return response


# 테스트 질문
test_questions = [
    "2019년 2월 전력수요 최대부하는 얼마인가요?",
    "전력수요 예측에서 기상 요인의 영향을 설명해주세요.",
    "What is the peak electricity demand in summer?"
]

print("=" * 80)
print("학습 전 모델 응답")
print("=" * 80)

for i, q in enumerate(test_questions, 1):
    print(f"\n[{i}] 질문: {q}")
    print(f"답변: {test_model(q, max_new_tokens=150)}")
    print("-" * 80)

학습 전 모델 응답

[1] 질문: 2019년 2월 전력수요 최대부하는 얼마인가요?
답변: 죄송합니다만, 저는 실시간 데이터 접근이 불가능하므로 2019년 2월의 전력수요 최대부하에 대한 정확한 정보를 제공하기는 어렵습니다. 이 정보는 지역 전력 회사 또는 에너지 통계 기관에서 제공할 수 있습니다. 일반적으로 이런 정보는 각국의 에너지 부처 웹사이트나 전력 공급사에서 확인 가능합니다. 필요한 경우 특정 국가의 전력 공급사 웹사이트를 방문하여 관련 정보를 확인해 보시기 바랍니다.
--------------------------------------------------------------------------------

[2] 질문: 전력수요 예측에서 기상 요인의 영향을 설명해주세요.
답변: 전력 수요 예측에서 기상 요인은 중요한 역할을 합니다. 기상 조건은 사람들의 활동 패턴과 에너지 사용에 직접적인 영향을 미칩니다. 다음은 기상 요인의 주요 영향 요소들입니다:

1. **기온**: 고온 또는 저온은 전력 수요를 크게 변화시킵니다. 일반적으로 낮은 온도에서는 가정이나 건물 내에서 더 많은 에어컨이 작동하여 전력 사용량이 증가합니다. 반대로 높은 온도에서는 냉방 시설이 작동하여 전력을 많이 소비하게 됩니다.

2.
--------------------------------------------------------------------------------

[3] 질문: What is the peak electricity demand in summer?
답변: Peak electricity demand in summer is often attributed to higher usage of air conditioning systems due to hot temperatures. In many regions, this peak demand can occur during the afternoon when people return home from work and

## 5. LoRA 설정

In [11]:
# LoRA 설정
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

model.print_trainable_parameters()


trainable params: 10,092,544 || all params: 7,625,709,056 || trainable%: 0.1323


## 6. 데이터 포맷팅 함수

In [12]:
ex = dataset["train"][0]
print(type(ex))                 # dict
print(type(ex["messages"]))     # list
print(formatting_func(ex))      # str


<class 'dict'>
<class 'list'>


NameError: name 'formatting_func' is not defined

In [13]:
def formatting_func(example):
    return tokenizer.apply_chat_template(
        example["messages"],   # ❗ 그대로 전달
        tokenize=False,
        add_generation_prompt=False,
    )

# 샘플 확인
sample = dataset['train'][0]
print("=== 포맷팅 전 ===")
print(sample['messages'])
print("\n=== 포맷팅 후 ===")
print(formatting_func({'messages': [sample['messages']]})[0][:300])

=== 포맷팅 전 ===
[{'role': 'user', 'content': 'What is the probability of temperatures being higher than normal in June 2022?'}, {'role': 'assistant', 'content': 'There is a 40% probability of temperatures being higher than normal in June 2022.'}]

=== 포맷팅 후 ===
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the probability of temperatures being higher than normal in June 2022?<|im_end|>
<|im_start|>assistant
There is a 40% probability of temperatures being higher than normal in Jun


## 7. SFT 학습 설정

In [14]:
from transformers import TrainingArguments

# 학습 파라미터
training_args = TrainingArguments(
    output_dir="./power_demand_sft_output",
    num_train_epochs=3,
    per_device_train_batch_size=2,  # GPU 메모리에 맞게 조정
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_steps=50,
    eval_steps=50,
    logging_steps=10,
    save_total_limit=3,
    warmup_steps=10,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    
    # ▼▼▼ 수정된 부분 (evaluation_strategy -> eval_strategy) ▼▼▼
    eval_strategy="steps", 
    # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
    
    load_best_model_at_end=True,
    report_to="none",  # wandb 사용 시 "wandb"로 변경
)

print("✓ Training arguments configured")

✓ Training arguments configured


In [15]:
import trl
print(trl.__version__)


0.26.2


In [ ]:


from trl import SFTTrainer, SFTConfig

# 1. SFTConfig
sft_config = SFTConfig(
    output_dir="./power_demand_sft_output",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    save_steps=50,
    eval_steps=50,
    logging_steps=10,
    save_total_limit=3,
    warmup_steps=10,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    load_best_model_at_end=True,
    report_to="none",
)


print("✓ SFT Config configured")

# 2. SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,   # ✅ tokenizer는 여기
    formatting_func=formatting_func,
)

print("✓ Trainer created")

✓ SFT Config configured
✓ Trainer created


In [17]:
model = model.to(dtype=torch.float16)
model.config.torch_dtype = torch.float16
model.config.use_cache = False

## 8. SFT 학습 실행

In [18]:
print("Starting SFT training...")
print(f"Total training samples: {len(dataset['train'])}")
print(f"Total eval samples: {len(dataset['test'])}")

effective_batch_size = (
    sft_config.per_device_train_batch_size
    * sft_config.gradient_accumulation_steps
)

print(f"Effective batch size: {effective_batch_size}")

trainer.train()

print("\n✓ Training completed!")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Starting SFT training...
Total training samples: 1025
Total eval samples: 114
Effective batch size: 8


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,0.469900,0.488842,0.478886,34105.000000,0.850611
100,0.422100,0.440628,0.445243,68475.000000,0.858440
150,0.352300,0.422178,0.399563,102443.000000,0.866042
200,0.363200,0.408742,0.388732,136613.000000,0.869839
250,0.360500,0.399079,0.377015,170883.000000,0.873807
300,0.337200,0.397866,0.351923,204477.000000,0.873293
350,0.331400,0.393987,0.355785,239506.000000,0.872360



✓ Training completed!


## 9. 모델 저장

In [19]:
# 학습된 모델 저장
output_dir = "./power_demand_sft_model"

trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ Model saved to {output_dir}")

✓ Model saved to ./power_demand_sft_model


## 10. 학습 후 모델 테스트

In [20]:
# 학습 후 테스트
print("=" * 80)
print("학습 후 모델 응답")
print("=" * 80)

for i, q in enumerate(test_questions, 1):
    print(f"\n[{i}] 질문: {q}")
    print(f"답변: {test_model(q, max_new_tokens=150)}")
    print("-" * 80)

학습 후 모델 응답

[1] 질문: 2019년 2월 전력수요 최대부하는 얼마인가요?
답변: 2019년 2월 전력수요 최대부하는 8,650만 kW로 예측되었습니다.
--------------------------------------------------------------------------------

[2] 질문: 전력수요 예측에서 기상 요인의 영향을 설명해주세요.
답변: 기상 요인은 전력 수요에 큰 영향을 줄 수 있습니다. 예를 들어, 날씨가 춥거나 덥고 습할 경우, 가정과 비즈니스에서 더 많은 에너지를 사용하게 됩니다. 이는 주로 공기 조건과 같은 기상 요인에 의해 발생합니다.
--------------------------------------------------------------------------------

[3] 질문: What is the peak electricity demand in summer?
답변: The peak electricity demand in summer (June to August) is projected to be 90,400 MW.
--------------------------------------------------------------------------------


In [25]:
# 추가 테스트 질문
additional_questions = [
    "여름철 전력수요가 높은 이유는?",
    "2023년 최대 전력수요는 언제 발생했나요?",
    "What are the main factors affecting power demand?",
    "2025년 1월의 평균 전력수요는 어떻게 될 것으로 생각하나요?",
    "평년 대비 전력수요 증가율을 어떻게 계산하나요?",
]

print("\n" + "=" * 80)
print("추가 테스트")
print("=" * 80)

for i, q in enumerate(additional_questions, 1):
    print(f"\n[{i}] 질문: {q}")
    print(f"답변: {test_model(q, max_new_tokens=200)}")
    print("-" * 80)


추가 테스트

[1] 질문: 여름철 전력수요가 높은 이유는?
답변: 여름철 전력수요는 날씨가 더운데도 불구하고 기온이 상대적으로 낮게 예측되거나, 대량의 폭염이 예상될 때 높을 가능성이 있습니다.
--------------------------------------------------------------------------------

[2] 질문: 2023년 최대 전력수요는 언제 발생했나요?
답변: 2023년 최대 전력수요는 9월에 85,700㎿로 발생했습니다.
--------------------------------------------------------------------------------

[3] 질문: What are the main factors affecting power demand?
답변: The main factors affecting power demand include weather conditions, economic growth, and public awareness of energy efficiency.
--------------------------------------------------------------------------------

[4] 질문: 2025년 1월의 평균 전력수요는 어떻게 될 것으로 생각하나요?
답변: 2025년 1월의 평균 전력수요는 67,800㎿로 예상됩니다.
--------------------------------------------------------------------------------

[5] 질문: 평년 대비 전력수요 증가율을 어떻게 계산하나요?
답변: 전력수요 증가율은 평년 대비 전력 수요의 변화를 나타내며, 2024년 1월 평년 대비 증가율은 6.8%로 예상됩니다.
--------------------------------------------------------------------------------


## 11. 성능 평가

In [26]:
# Test set 샘플로 성능 평가
test_samples = dataset['test'].select(range(min(10, len(dataset['test']))))

print("=" * 80)
print("Test Set 샘플 평가")
print("=" * 80)

for i, sample in enumerate(test_samples, 1):
    question = sample['messages'][0]['content']
    expected = sample['messages'][1]['content']
    predicted = test_model(question, max_new_tokens=200)
    
    print(f"\n[{i}] 질문: {question}")
    print(f"\n정답: {expected}")
    print(f"\n예측: {predicted}")
    print("-" * 80)

Test Set 샘플 평가

[1] 질문: What is the forecast for weekly maximum electricity demand in October 2024?

정답: The weekly maximum electricity demand in October 2024 is forecasted as follows: Week 1: 77,000 MW, Week 2: 73,400 MW, Week 3: 73,000 MW, Week 4: 72,700 MW, Week 5: 74,900 MW.

예측: The forecast for weekly maximum electricity demand in October 2024 ranges from 76,100 MW to 81,700 MW across four weeks.
--------------------------------------------------------------------------------

[2] 질문: 전력 수요는 보통 어떤 계절에 가장 높나요?

정답: 전력 수요는 일반적으로 겨울과 여름에 가장 높고, 봄과 가을에는 낮은 경향이 있습니다.

예측: 전력 수요는 여름(7-8월)에 가장 높으며, 겨울(12-2월)에 차순으로 높습니다.
--------------------------------------------------------------------------------

[3] 질문: 2024년 10월의 전력 수요 전망 결과는 어떻게 되나요?

정답: 2024년 10월의 전력 수요 전망 결과는 60,200㎿로, 여러 모델의 평균을 반영한 수치입니다.

예측: 2024년 10월의 전력 수요는 69,500 MW로 예상됩니다.
--------------------------------------------------------------------------------

[4] 질문: What was the highest electricity demand recorded in August

## 12. 결론

### SFT 접근법의 장점:
- ✅ Instruction-following 능력 유지
- ✅ 전력수요 도메인 지식 학습
- ✅ 질문-답변 형태로 즉시 사용 가능

### 다음 단계:
1. **더 많은 데이터**: GPT API로 전체 125개 파일 처리
2. **하이퍼파라미터 튜닝**: learning rate, epochs 조정
3. **평가 메트릭**: BLEU, ROUGE 등으로 정량 평가
4. **배포**: FastAPI 등으로 서빙

### 비용 고려:
- GPT-4o-mini: 약 $0.15/1M input tokens
- 125개 파일 × 평균 5000 tokens = 625K tokens
- 예상 비용: ~$0.10 (매우 저렴)